In [1]:
import merlin.models.tf as ml
from merlin.datasets.synthetic import generate_data

ecommerce_data = generate_data("e-commerce", num_rows=1000)

ecommerce_data.to_dff()

2022-02-17 10:26:19.825592: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-17 10:26:21.325490: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 45041 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6
2022-02-17 10:26:21.326887: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 45586 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:21:00.0, compute capability: 8.6


,user_id,user_categories,user_shops,user_brands,user_intentions,user_profile,user_group,user_gender,user_age,user_consumption_1,...,item_shop,item_intention,item_brand,click,conversion,user_item_categories,user_item_shops,user_item_brands,user_item_intentions,position
0,21,11,18,9,2,1,1,1,1,1,...,1,1,1,1,0,64,492,18,22,3
1,3,2,2,1,1,1,1,1,1,1,...,10,1,5,0,1,97,226,17,24,1
2,5,3,4,2,1,1,1,1,1,1,...,32,2,16,0,1,149,499,174,10,3
3,4,2,3,2,1,1,1,1,1,1,...,6,1,3,1,1,214,279,161,7,3
4,54,28,47,24,5,2,2,1,1,1,...,4,1,2,1,0,110,49,130,13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,16,8,14,7,2,1,1,1,1,1,...,3,1,2,0,0,185,312,128,2,3
996,63,33,55,28,6,3,2,1,1,1,...,10,1,5,1,0,235,179,223,13,3
997,9,5,8,4,1,1,1,1,1,1,...,6,1,3,1,0,257,95,172,23,3
998,10,5,8,4,1,1,1,1,1,1,...,39,2,20,1,0,215,363,81,24,1


## Retrieval

In [13]:
retrieval_model = ml.TwoTowerModel(ecommerce_data.schema, query_tower=ml.MLPBlock([512, 256]))

retrieval_model.compile(optimizer="adam", run_eagerly=True)
retrieval_model.fit(ecommerce_data, batch_size=50, epochs=2)

print(f"Number of parameters: {retrieval_model.count_params()}")

The sampler InBatchSampler returned no samples for this batch.


Epoch 1/2
20/20 [==============================] - 4s 210ms/step - ndcg_at_1_10: 0.0895 - ndcg_at_1_20: 0.1419 - recall_at_1_10: 0.1952 - recall_at_1_20: 0.4038 - avg_precision_at_1_10: 0.0585 - avg_precision_at_1_20: 0.0727 - loss: 3.8956 - regularization_loss: 0.0000e+00 - total_loss: 3.8956
Epoch 2/2
20/20 [==============================] - 4s 212ms/step - ndcg_at_1_10: 0.2134 - ndcg_at_1_20: 0.2711 - recall_at_1_10: 0.3895 - recall_at_1_20: 0.6190 - avg_precision_at_1_10: 0.1606 - avg_precision_at_1_20: 0.1763 - loss: 3.8606 - regularization_loss: 0.0000e+00 - total_loss: 3.8606
Number of parameters: 1113010


## Ranking

In [11]:
ranking_model = ml.DLRMModel(
    ecommerce_data.schema,
    embedding_dim=64,
    bottom_block=ml.MLPBlock([64]),
    top_block=ml.MLPBlock([32]),
)

ranking_model.compile(optimizer="adam", run_eagerly=True)
ranking_model.fit(ecommerce_data, batch_size=50, epochs=2)

print(f"Number of parameters: {ranking_model.count_params()}")

Epoch 1/2
20/20 [==============================] - 1s 60ms/step - click/binary_classification_task/precision: 0.5180 - click/binary_classification_task/recall: 0.8117 - click/binary_classification_task/binary_accuracy: 0.5140 - click/binary_classification_task/auc: 0.4885 - conversion/binary_classification_task/precision: 0.5022 - conversion/binary_classification_task/recall: 0.4598 - conversion/binary_classification_task/binary_accuracy: 0.5040 - conversion/binary_classification_task/auc: 0.4985 - loss: 0.6944 - regularization_loss: 0.0000e+00 - total_loss: 0.6944
Epoch 2/2
20/20 [==============================] - 1s 60ms/step - click/binary_classification_task/precision: 0.5871 - click/binary_classification_task/recall: 0.8641 - click/binary_classification_task/binary_accuracy: 0.6170 - click/binary_classification_task/auc: 0.6516 - conversion/binary_classification_task/precision: 0.6362 - conversion/binary_classification_task/recall: 0.5863 - conversion/binary_classification_task/bi